# Image Slice inferencing
Testing image slice inferenceing for microservice setup

# 1) Imports and mount

In [1]:
%load_ext autoreload
%autoreload 2

# This sets up the appropriate logging and path configs
from notebook_setup import * 

In [2]:
import torch
from torchsummary import summary
import matplotlib.pyplot as plt

from monai.losses import DiceLoss
from src.features.build_features import train_transform, val_transform, test_transform

from src.visualization.visualize import view_slice

from monai.data import Dataset, DataLoader
from src.enums import DataDict

from src.settings.config import get_app_settings
from src.utils import load_yaml

## 2) Import Settings

In [3]:
settings = get_app_settings()
model_config = load_yaml("model_configs.yaml")

In [13]:
from src.enums import INTERIM_TESTING_DATA_PATHS, INTERIM_TRAINING_DATA_PATHS
from src.pytorch_utils import get_interim_data_path


train_paths = get_interim_data_path(INTERIM_TRAINING_DATA_PATHS)
val_paths = get_interim_data_path(INTERIM_TESTING_DATA_PATHS)

train_dataset  = Dataset(train_paths, train_transform)
validation_dataset  = Dataset(val_paths, val_transform)
test_dataset = Dataset(val_paths, test_transform)

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE_TRAIN, shuffle=True, num_workers=2)
# validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE_VAL, shuffle=True)

# from monai.transforms import (LoadImaged)
# image_loader = LoadImaged(keys=[DataDict.ImageT1, DataDict.ImageFlair])
# image_loader(val_paths)
test_dataset.__getitem__(0)[DataDict.ImageT1]

metatensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

## 3) Inference



### 3.1) Infernce on volume nii

In [6]:
# Get sample volume 
from src.data.make_dataset import get_test_raw_paths

raw_test_paths = get_test_raw_paths()
raw_test_paths = get_interim_data_path(INTERIM_TESTING_DATA_PATHS)
# train_paths = get_interim_data_path(INTERIM_TRAINING_DATA_PATHS)
# val_paths = get_interim_data_path(INTERIM_TESTING_DATA_PATHS)

In [7]:
from copy import copy
sample = copy(raw_test_paths[0])
import nibabel as nib
import numpy as np
from src.pytorch_utils import slice_tensor_volume
# Lets load it as a nib file as this is how it will come from the front end
# t1_sample = nib.load(sample[DataDict.ImageT1])
# flair_sample = nib.load(sample[DataDict.ImageFlair])
# label_sample = nib.load(sample[DataDict.Label])
t1_sample = torch.tensor([nib.load(sample[DataDict.ImageT1]).get_fdata()])
flair_sample =  torch.tensor([nib.load(sample[DataDict.ImageFlair]).get_fdata()])
label_sample =  torch.tensor([nib.load(sample[DataDict.Label]).get_fdata()])

# t1_sample = torch.Tensor([nib.Nifti1Image(slice_tensor_volume(t1_sample)[0], affine=np.eye(4)).get_fdata()])
# flair_sample = torch.Tensor([nib.Nifti1Image(slice_tensor_volume(flair_sample)[0], affine=np.eye(4)).get_fdata()])
# label_sample = torch.Tensor([nib.Nifti1Image(slice_tensor_volume(label_sample)[0], affine=np.eye(4)).get_fdata()])

# Set it in the sample
sample[DataDict.ImageT1] = t1_sample
sample[DataDict.ImageFlair] = flair_sample
sample[DataDict.Label] = label_sample

C:\Users\zifen\AppData\Local\Temp\ipykernel_16400\1108633950.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  t1_sample = torch.tensor([nib.load(sample[DataDict.ImageT1]).get_fdata()])


In [8]:
sample[DataDict.ImageT1].size

<function Tensor.size>

In [9]:
sample_data_paths = [sample]

In [12]:

from monai.transforms import (
    Compose,
    ToTensord, LoadImaged, EnsureChannelFirstd, ToMetaTensord, Spacingd,Resized
)
from src.features.transforms import ImagesToMultiChannel

custom_transform = Compose(
    [
        ToMetaTensord(keys=[DataDict.ImageT1, DataDict.ImageFlair]),
        EnsureChannelFirstd(
            keys=[DataDict.ImageT1, DataDict.ImageFlair],
            channel_dim=0
        ),
        # ConvertToMultiChannelBasedOnLabelsClassesd(keys=[DataDict.Label]),
        # Spacingd(
        #     keys=[DataDict.ImageT1, DataDict.ImageFlair],
        #     pixdim=(1.5, 1.5, 2.0),
        #     mode=("bilinear", "bilinear", "nearest"),
        # ),
        # Resized(
        #     keys=[DataDict.ImageT1, DataDict.ImageFlair],
        #     spatial_size=[256, 256],
        # ),
        # Orientationd(
        #     keys=[DataDict.ImageT1, DataDict.ImageFlair, DataDict.Label], axcodes="RAS"
        # ),
        # ToTensord(keys=[DataDict.ImageT1, DataDict.ImageFlair]),
        # ImagesToMultiChannel(keys=[DataDict.ImageT1, DataDict.ImageFlair]),
    ]
)

test_dataset = Dataset(sample_data_paths, custom_transform)
test_dataset.__getitem__(0)[DataDict.ImageT1]
# view_slice(test_dataset.__getitem__(0)[DataDict.ImageT1][..., 30])

metatensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], dtype=torch.float64)

In [ ]:

from monai.transforms import (
    EnsureChannelFirstd,
    Compose,
    LoadImaged,
    Resized,
    Spacingd,
    ToTensord,
    Orientationd,
    RandFlipd,
    RandScaleIntensityd,
    RandShiftIntensityd,
    Rand2DElasticd,
    RandZoomd,
    ToMetaTensord,
)
inference_transform = Compose(
    [
        LoadImaged(keys=[DataDict.ImageT1, DataDict.ImageFlair]),
        EnsureChannelFirstd(keys=[DataDict.ImageT1, DataDict.ImageFlair]),
        # ToMetaTensord(keys=[DataDict.ImageT1, DataDict.ImageFlair, DataDict.Label])
    ]
)

from src.features.build_features import inference_transform
test_dataset = Dataset(raw_test_paths, inference_transform)
test_dataset.__getitem__(0)

RuntimeError: applying transform <monai.transforms.compose.Compose object at 0x000002267DD16F50>

In [ ]:
from src.features.build_features import inference_transform
test_dataset = Dataset(sample_data_paths, inference_transform)

In [ ]:
subject_data = test_dataset.__getitem__(0)
subject_data

In [ ]:
from src.models.predict_model import ImagePredictor
from src.models.train_model import model

pred_network = model
checkpoint = torch.load('models/single_slice_t1_flair_v1.pt', map_location=torch.device('cpu'))
pred_network.load_state_dict(checkpoint['model_state_dict'])
img_predictor = ImagePredictor(pred_network, test_dataset)
test_predictions = img_predictor.predict_handler()
# print('DONE')

In [ ]:
from monai.inferers import inferer
from monai.transforms import Compose
model.eval()
device = torch.device("cpu")
transform = Compose([
    ToTensor(), 
    LoadImage(image_only=True)
    ])
data = transform(sample_data_paths).to(device)
with torch.no_grad():
    pred = inferer(inputs=data, network=model)